<a href="https://colab.research.google.com/github/vndee/pytorch-vi/blob/master/chatbot_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CHATBOT
**Tác giả**: [Matthew Inkawhich](https://github.com/MatthewInkawhich)

Trong hướng dẫn này chúng ta sẽ khám phá một ứng dụng thú vị của mô hình seq2seq. Chúng ta sẽ huấn luyện một chatbot đơn giản sử dụng data là lời thoại trong phim từ [Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)

Các mô hình có khả năng đàm thoại là một mảng nghiên cứu đang rất được chú ý của trí tuệ nhân tạo. Chatbot có thể tìm thấy trong rất nhiều sản phẩm tiện ích như bộ phận chăm sóc khách hàng hoặc các dịch vụ tư vấn online. Nhưng con bot này thường thuộc dạng retrieval-based (dựa trên truy xuất), đó là các mô hình mà câu trả lời đã được định sẵn cho mỗi loại câu hỏi nhất định. Dạy một cỗ máy để nó có khả năng đàm thoại với con người một cách tự nhiên vẫn là một bài toán khó và còn xa để đi đến lời giải. Gần đây, đi theo sự bùng nổ của học sâu, các mô hình sinh mạnh mẽ như Google's Neural Conversational Model đã tạo ra một bước nhảy vọt ấn tượng. Trong bài hướng dẫn này, chúng ta sẽ hiện thực một kiểu mô hình sinh như vậy với PyTorch.

![](https://pytorch.org/tutorials/_images/bot.png)

```
> hello?
Bot: hello .
> where am I?
Bot: you re in a hospital .
> who are you?
Bot: i m a lawyer .
> how are you doing?
Bot: i m fine .
> are you my friend?
Bot: no .
> you're under arrest
Bot: i m trying to help you !
> i'm just kidding
Bot: i m sorry .
> where are you from?
Bot: san francisco .
> it's time for me to leave
Bot: i know .
> goodbye
Bot: goodbye .
```

### Các phần chính:
- Load và tiền xử lý [Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) dataset.
- Hiện thực mô hình seq2seq với Luong's attention.
- Phối hợp huấn luyện mô hình encoder-decoder với mini-batches.
- Hiện thực thuật toán decoding bằng tìm kiếm tham lam.
- Tương tác với mô hình đã huấn luyện.

### Lời cảm ơn:
Code trong bài viết này được mượn từ các project mã nguồn mở sau:
- Yuan-Kuei Wu’s pytorch-chatbot implementation: https://github.com/ywk991112/pytorch-chatbot
- Sean Robertson’s practical-pytorch seq2seq-translation example: https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation
- FloydHub’s Cornell Movie Corpus preprocessing code: https://github.com/floydhub/textutil-preprocess-cornell-movie-corpus

## Chuẩn bị
Đầu tiên chúng ta cần tải dữ liệu tại [đây](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) và giải nén.

In [1]:
!wget --header 'Host: www.cs.cornell.edu' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip' --output-document 'cornell_movie_dialogs_corpus.zip'
!unzip cornell_movie_dialogs_corpus.zip

--2019-05-18 16:05:44--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  2.44MB/s    in 7.3s    

2019-05-18 16:05:52 (1.30 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-

In [3]:
!ls cornell\ movie-dialogs\ corpus

chameleons.pdf		       movie_lines.txt		  README.txt
movie_characters_metadata.txt  movie_titles_metadata.txt
movie_conversations.txt        raw_script_urls.txt


Import một số thư viện hỗ trợ:

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

## Load và tiền xử lý dữ liệu
Bước tiếp theo chúng ta cần tổ chức lại dữ liệu. Cornell Movie-Dialogs Corpus là một tập dữ liệu lớn gồm các đoạn hội thoại của các nhân vật trong phim.
- 220,579 đoạn hội thoại của 10,292 cặp nhân vật.
- 9,035 nhân vật từ 617 bộ phim.
- 304,713 cách diễn đạt.

Tập dữ liệu này rất lớn và phân tán, đa dạng trong phong cách ngôn ngữ, thời gian, địa điểm cũng như ý nghĩa. Chúng ta hi vọng mô hình của mình sẽ đủ tốt để làm việc với nhiều cách nói hay truy vấn khác nhau.
Trước hết, hãy xem một vài dòng từ dữ liệu gốc, xem chúng ta có gì ở đây.

In [6]:
corpus_name = 'cornell movie-dialogs corpus'

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus_name, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


Để thuận tiện, chúng ta sẽ tổ chức lại dữ liệu theo một format mỗi dòng trong file sẽ được tách ra bởi dấu tab cho một câu hỏi và một câu trả lời.

Phía dưới chúng ta sẽ cần một số phương thức để phân tích dữ liệu từ file movie_lines.tx
- `loadLines': Tách mỗi dòng dữ liệu thành một đối tượng dictionary trong python gồm các thuộc tính (lineID, characterID, movieID, character, text).
-`loadConversations`: Nhóm các thuộc tính của từng dòng trong `loadLines` thành một đoạn hội thoại dựa trên movie_conversations.txt.
- `extractSentencePairs`: Trích xuất một cặp câu trong đoạn hội thoại.

In [0]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            lineIds = eval(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs


Bây giờ chúng ta sẽ gọi các phương thức ở trên để tạo ra một file dữ liệu mới tên là formatted_movie_lines.txt.

In [10]:
# Define path to new file
datafile = os.path.join(corpus_name, 'formatted_movie_lines.txt')

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus_name, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus_name, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

### Đọc và cắt dữ liệu
Sau khi đã tổ chức lại dữ liệu, chúng ta cần tạo một từ điển các từ dùng trong tập dữ liệu và đọc các cặp câu truy vấn - phản hồi vào bộ nhớ.

Chú ý rằng chúng ta xem một câu là một chuỗi liên tiếp các **từ**, không có một ánh xạ ngầm nào của nó ở một không gian số học rời rạc. Do đó chúng ta cần phải tạo một hàm ánh xạ sao cho mỗi từ riêng biệt chỉ có duy nhất một giá trị chỉ số đại diện chính là vị trí của nó trong từ điển.

Để làm điều đó chúng ta định nghĩa lớp `Voc`, nơi sẽ lưu một dictionary ánh xạ **từ** sang **chỉ số**, một dictionary ánh xạ ngược **chỉ số** sang **từ**, một biến đếm cho mỗi từ và một biến đếm tổng số các từ. Lớp `Voc` cũng cung cắp các phương thức để thêm một từ vào từ điển (`addWord`), thêm tất cả các từ trong một câu (`addSentence`) và lược bỏ (trimming) các từ không thường gặp. Chúng ta sẽ nói về trimming sau:


In [0]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Trước khi đưa vào huấn luyện ta cần một số thao tác tiền xử lý dữ liệu. Đầu tiên, chúng ta cần chuyển đổi các chuỗi Unicode thành ASCII sử dụng `unicodeToAscii`. Tiếp theo phải chuyển tất cả các kí tự thành chữ viết thường và lược bỏ các kí tự không ở trong bảng chữ cái ngoại trừ một số dấu câu (`normalizedString`). Cuối cùng để giúp quá trình huấn luyện nhanh chóng hội tụ chúng ta sẽ lọc ra các câu có độ dài lớn hơn ngưỡng `MAX_LENGTH` (`filterPairs`).

In [14]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("save")
voc, pairs = loadPrepareData(corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']
